In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyomo.opt import SolverFactory, TerminationCondition
from src.analyses.value_function import ValueFunction
from src.analyses.convergence_analysis import HausdorffAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from scipy.optimize import curve_fit
from NSPLIB.src.instances.process.process_SP import const_model
import copy
# Constants
process_obj = -1126.4218270121305
process_sol = {"x1":1727.2601809997955,"x2":16000,"x3":104.23841082714829,"x5":2000}
process_y_bound = {"x1":[10,2000],"x2":[0,16000],"x3":[0,120],"x5":[0,2000]}
process_y_bound = {"x1":[1725,1730],"x2":[15999,16000],"x3":[100,110],"x5":[1999,2000]}
# Build model
sto_m = const_model()
m = CaoZavalaModel.from_sto_m(sto_m)
m.build()

v_f= ValueFunction(m,solver="baron")
y_dimension = {'x1':{}, 'x2':{}, 'x3':{}, 'x5':{}}
steps=10
for y in y_dimension.keys():
    v_f.calc_1D(idx=y, y_val_fix=copy.deepcopy(process_sol),
                    interval=process_y_bound[y], step=steps)
    for s in m.scenarios:
        y_dimension[y][s] = v_f.value_func[s]
    y_dimension[y]['total'] = v_f.total_value_func


# Specify grid dimensions
n_rows = len(y_dimension)
n_cols = len(m.scenarios) + 1  # +1 for 'total'

# Create a grid of Output widgets for the plots
outputs = [[widgets.Output() for _ in range(n_cols)] for _ in range(n_rows)]

# Fill each plot with the corresponding value function data
for i, y_key in enumerate(y_dimension.keys()):
    for j in range(n_cols):
        with outputs[i][j]:
            x = np.linspace(process_y_bound[y_key][0], process_y_bound[y_key][1], len(y_dimension[y_key]['total']))
            if j < len(m.scenarios):
                scenario_key = list(m.scenarios)[j]
                y_vals = y_dimension[y_key][scenario_key]
                title = f"{y_key} - Scenario {scenario_key}"
            else:
                y_vals = y_dimension[y_key]['total']
                title = f"{y_key} - Total"
            fig, ax = plt.subplots(figsize=(4,4))
            ax.plot(x, y_vals,marker="o")
            ax.set_title(title)
            ax.set_xlabel(y_key)
            ax.set_ylabel("Objective Value")
            ax.tick_params(axis='both', labelsize=14)
            ax.xaxis.label.set_size(16)
            ax.yaxis.label.set_size(16)
            ax.title.set_size(18)
            plt.tight_layout()
            plt.show()

# Arrange the outputs in a grid
grid = widgets.GridBox(
    [outputs[i][j] for i in range(n_rows) for j in range(n_cols)],
    layout=widgets.Layout(grid_template_columns=f"repeat({n_cols}, 400px)")
)

display(grid)

GridBox(children=(Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Ou…